In [ ]:
!pip install tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


def ex_tag(sid, page):
    """
    뉴스 분야(sid)와 페이지(page)를 입력하면 해당 기사 링크들을 리스트로 추출하는 함수
    """
    # 1. URL 설정
    url = f"https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1={sid}&page={page}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
    }
    html = requests.get(url, headers=headers)

    # 2. HTML 파싱
    soup = BeautifulSoup(html.text, "lxml")

    # 3. 기사 목록에서 링크 추출
    tag_lst = []
    news_list = soup.select('.type06_headline li dl a')  # 기사 링크가 포함된 a 태그 선택

    for a in news_list:
        href = a.get("href")
        if href and "article" in href:
            tag_lst.append(href)

    return tag_lst
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(100)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)

    return re_lst

all_hrefs = {}
sids = [101]  # 분야 리스트

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

In [ ]:
all_hrefs

In [ ]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수

    Args:
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스

    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리

    """
    art_dic = {}

    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    url = all_hrefs[101][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/130.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")

    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)

    # 날짜 수집
    # date = soup.select(date_selector)
    # date_lst = [d.text for d in date]
    # date_str = "".join(date_lst)

    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)

    ## 3.
    art_dic["title"] = title_str
    # art_dic["date"] = date_str
    art_dic["main"] = main_str

    return art_dic

In [ ]:
section_lst = [101]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)


In [ ]:
artdic_lst

In [ ]:
import pandas as pd
import json

def csv_to_custom_json(csv_file: str, json_file: str):
    """
    CSV 파일에서 'title' 열을 추출하여 'relation_type'에 삽입하고 JSON 파일로 저장.

    Args:
        csv_file (str): 원본 CSV 파일 경로.
        json_file (str): 저장할 JSON 파일 경로.
    """
    try:
        # CSV 파일 읽기
        df = pd.read_csv(csv_file)

        # 'title' 열 추출
        if 'title' not in df.columns:
            raise ValueError("'title' 열이 CSV 파일에 없습니다.")

        titles = df['title'].dropna().tolist()  # 'title' 열을 리스트로 변환

        # JSON 형식 변환
        result = []
        for title in titles:
            result.append({
                "relation_type": title,  # 'title' 값을 relation_type에 삽입
                "keywords": ["키워드1", "키워드2", "키워드3"]  # 임의의 키워드
            })

        # JSON 파일로 저장
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

        print(f"JSON 파일로 저장되었습니다: {json_file}")
    except Exception as e:
        print(f"에러 발생: {e}")

# 사용 예제
csv_file_path = "/article_df.csv"  # 원본 CSV 파일 경로
json_file_path = "/titles_with_relation_type.json"  # 저장할 JSON 파일 경로

csv_to_custom_json(csv_file_path, json_file_path)


In [ ]:
import pandas as pd
import re

# CSV 파일을 불러오는 함수 (한글 파일 읽기 지원)
def load_training_articles(file_path):
    try:
        # UTF-8로 먼저 시도
        df = pd.read_csv(file_path, usecols=['title', 'main'], encoding='utf-8')
    except UnicodeDecodeError:
        # UTF-8로 읽지 못하면 EUC-KR로 재시도
        df = pd.read_csv(file_path, usecols=['title', 'main'], encoding='euc-kr')

    # main 열에서 .com, co.kr, 전화번호 제거
    def clean_text(text):
        # .com, co.kr 제거
        text = re.sub(r'\b\w+\.(com|co\.kr)\b', '', text)
        # 전화번호 형식 제거 (010-1234-5678, (02) 1234-5678 등)
        text = re.sub(r'\b\d{2,3}[-.\)]?\d{3,4}[-.]\d{4}\b', '', text)
        return text

    # 정규 표현식을 통해 불필요한 부분 제거
    df['main'] = df['main'].apply(clean_text)

    # title과 main을 결합하여 하나의 리스트로 생성
    training_articles = df.apply(lambda row: f"{row['title']} {row['main']}", axis=1).tolist()

    return training_articles

# 예시 파일 경로
file_path = 'article_df.csv'

# training_articles 생성
training_articles = load_training_articles(file_path)

# 결과 확인
for article in training_articles:
    print("Article:", article)
    print()  # 각 기사 간 개행
